In [19]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from glob import glob

In [20]:
# Check the number of GPUs available for acceleration
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 2


In [21]:
model = tf.saved_model.load('/kaggle/input/movenet/tensorflow2/singlepose-thunder/4')
model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7da70f71e2c0>

In [26]:
def extract_keypoints(image_path):
    """
    Process an image, run inference with MoveNet, and return keypoints.
    """
    # Load and preprocess the image
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, (256, 256))  # Resize to model's input size
    img = tf.cast(img, dtype=tf.int32)  # Ensure datatype matches model requirements
    img = tf.expand_dims(img, axis=0)  # Add batch dimension

    # Run inference
    outputs = model.signatures['serving_default'](img)
    keypoints = outputs['output_0'].numpy()  # [1, 1, 17, 3]

    # Reshape to extract 17 keypoints (x, y, confidence)
    return keypoints.reshape(17, 3)


In [27]:
# Path to the dataset directory
dataset_dir = "/kaggle/input/pose-samples/dataset"

output_data = []

In [28]:
# Traverse through each class folder and process images
for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):  # Ensure it's a directory
        for image_path in glob(f"{class_path}/*.jpg"):
            keypoints = extract_keypoints(image_path)
            # Flatten keypoints and include class label
            row = [image_path, class_name] + keypoints.flatten().tolist()
            output_data.append(row)

# Create DataFrame with columns for file path, class name, and keypoints
columns = ["filepath", "label"] + [f"keypoint_{i}_{attr}" for i in range(1, 18) for attr in ["x", "y", "confidence"]]
df = pd.DataFrame(output_data, columns=columns)

# Save to CSV
df.to_csv("pose_data.csv", index=False)